In [21]:
import pandas as pd
import numpy as np

# List of CSV files
csv_files = ['POI_FINAL_Museum.csv', 'POI_FINAL_Restaurant.csv', 'POI_FINAL_Travel_Agency.csv']

# Define weights for each criterion
weights = {
    'Distance': 0.2,
    'Popularity': 0.1,
    'Ratings': 0.3,
    'Total Ratings': 0.1,
    'Price($)': 0.3
}

# Define the pairwise comparison matrix for criteria
criteria_comparison = {
    'Distance': [1, 3, 5, 7, 9],
    'Popularity': [1/3, 1, 3, 5, 7],
    'Ratings': [1/5, 1/3, 1, 3, 5],
    'Total Ratings': [1/7, 1/5, 1/3, 1, 3],
    'Price($)': [1/9, 1/7, 1/5, 1/3, 1]
}

for file in csv_files:
    # Read the CSV file
    data = pd.read_csv(file, encoding='latin1')
    
    # Normalize the data
    for criterion in weights.keys():
        if criterion == 'Price($)':
            data[criterion] = (data[criterion].max() - data[criterion]) / (data[criterion].max() - data[criterion].min())
        else:
            data[criterion] = (data[criterion] - data[criterion].min()) / (data[criterion].max() - data[criterion].min())
    
    # Calculate the weighted sum using AHP
    weighted_sum = np.zeros(len(data))
    for criterion, weight in weights.items():
        pairwise_comparison_vector = np.array(criteria_comparison[criterion])
        normalized_comparison_vector = pairwise_comparison_vector / np.sum(pairwise_comparison_vector)
        normalized_comparison_vector_extended = np.repeat(normalized_comparison_vector, len(data) // len(normalized_comparison_vector))
        weighted_sum += data[criterion].values * normalized_comparison_vector_extended * weight

    
    # Add the weighted sums to the DataFrame
    data['Weighted Sum'] = weighted_sum
    
    # Rank the alternatives based on the weighted sum
    data['Rank'] = data['Weighted Sum'].rank(ascending=False, method='min')
    
    # Display the result
    print(f"Results for {file}:")
    # Truncate the name
    data['Name'] = data['Name'].apply(lambda x: ' '.join(x.split()[:2]))
    print(data[['Name', 'Latitude', 'Longitude', 'Weighted Sum', 'Rank']])
    print("\n")


Results for POI_FINAL_Museum.csv:
                Name   Latitude  Longitude  Weighted Sum  Rank
0            Jane A.  40.767370 -73.981125      0.018420   8.0
1           Spyscape  40.765198 -73.983618      0.017915   9.0
2         The Africa  40.796289 -73.949378      0.012742  10.0
3    Madame Tussauds  40.756539 -73.988521      0.028508   7.0
4           New York  40.752895 -73.977589      0.071232   5.0
5         Center for  40.752768 -73.992304      0.039571   6.0
6  Fashion Institute  40.747304 -73.994797      0.169219   3.0
7       Hebrew Union  40.728807 -73.994549      0.136504   4.0
8         Lower East  40.724864 -73.992735      0.449736   1.0
9            The New  40.726021 -73.999809      0.309176   2.0


Results for POI_FINAL_Restaurant.csv:
                    Name   Latitude  Longitude  Weighted Sum  Rank
0       Daily Provisions  40.782225 -73.978646      0.023968   7.0
1          Levain Bakery  40.781511 -73.979099      0.013476  10.0
2            Sushi Kaito  40.779